# DESI Fisher examples

In this notebook we will run some DESI forecasts.

In [1]:
import numpy as np

zedges = np.linspace(1.1, 1.6, 6)
zranges = np.array(list(zip(zedges[:-1], zedges[1:])))
nbars = np.array([1079.9998, 1036.0009, 970.9995, 876.9994, 553.9985])  # nobj / deg^2 / dz
b0 = 0.84  # at z = 0

In [2]:
from cosmoprimo.fiducial import DESI
from desilike.theories.galaxy_clustering import (BAOPowerSpectrumTemplate, SimpleBAOWigglesTracerPowerSpectrumMultipoles, DampedBAOWigglesTracerPowerSpectrumMultipoles,
                                                 StandardPowerSpectrumTemplate, SimpleTracerPowerSpectrumMultipoles)
from desilike.observables.galaxy_clustering import TracerPowerSpectrumMultipolesObservable, CutskyFootprint, ObservablesCovarianceMatrix
from desilike.likelihoods.galaxy_clustering import ObservablesGaussianLikelihood, SNWeightedPowerSpectrumLikelihood
from desilike import Fisher, setup_logging

#setup_logging()

## Let's compare simple isotropic BAO forecasts against some reference

### SN-weighted version

In [3]:
cosmo = DESI()
fo = cosmo.get_fourier()

fishers = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = r * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb')  # prescription for linear bias
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    # Footprint, to get volume and shot noise; provided nbar is in deg^(-2) dz^(-1)
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))

    fisher = Fisher(likelihood)
    fishers.append(fisher(**params))

# Let's compare against reference: within a few percent, not too bad!
refs = [0.7019, 0.7107, 0.7296, 0.7658, 1.0062]
print(100. * np.array([fisher.std(params='qiso') for fisher in fishers]) / np.array(refs)[:len(fishers)])

/home/adematti/Bureau/DESI/NERSC/cosmodesi/cosmoprimo/cosmoprimo/eisenstein_hu.py:23: UserWarning: EisensteinHuNoWiggleEngine cannot cope with massive neutrinos
  warnings.warn('{} cannot cope with massive neutrinos'.format(self.__class__.__name__))


[1.10363831 1.10627461 1.11126587 1.11931011 1.14020594]


### Comparison against multipole compression

In [4]:
fishers2 = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = r * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    observable = TracerPowerSpectrumMultipolesObservable(data=params,  # data can be a dictionary of parameters
                                                         # fit monopole, quadrupole and hexadecapole between 0.01 and 0.5 h/Mpc, with 0.005 h/Mpc steps
                                                         klim={0: [0.01, 0.5, 0.005], 2: [0.01, 0.5, 0.005], 4: [0.01, 0.5, 0.005]},
                                                         theory=theory)
    covariance = ObservablesCovarianceMatrix(observable, footprints=footprint, resolution=5)  # Gaussian covariance matrix
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(**params))
    fisher = Fisher(likelihood)
    fishers2.append(fisher(**params))

# multipole compression is very close to optimal...
print(np.array([fisher.std(params='qiso') for fisher in fishers2]) / np.array([fisher.std(params='qiso') for fisher in fishers]))

/home/adematti/anaconda3/envs/cosmodesi/lib/python3.9/site-packages/numpy/lib/polynomial.py:1329: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


[1.0008864  1.00085707 1.00080833 1.00073431 1.00043191]


### Comparison against correlation function multipoles

In [5]:
from desilike.theories.galaxy_clustering import SimpleBAOWigglesTracerCorrelationFunctionMultipoles, DampedBAOWigglesTracerCorrelationFunctionMultipoles
from desilike.observables.galaxy_clustering import TracerCorrelationFunctionMultipolesObservable

fishers3 = []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = r * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b0 * fo.sigma8_cb/fo.sigma8_z(z, of='delta_cb'), 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qisoqap')
    #theory = DampedBAOWigglesTracerCorrelationFunctionMultipoles(template=template)
    theory = SimpleBAOWigglesTracerCorrelationFunctionMultipoles(template=template)
    observable = TracerCorrelationFunctionMultipolesObservable(data=params,  # data can be a dictionary of parameters
                                                               # fit monopole, quadrupole and hexadecapole
                                                               slim={0: [30., 170., 2], 2: [30., 170., 2], 4: [30., 170., 2]},
                                                               theory=theory)
    covariance = ObservablesCovarianceMatrix(observable, footprints=footprint, resolution=5)  # Gaussian covariance matrix
    likelihood = ObservablesGaussianLikelihood(observables=observable, covariance=covariance(**params))
    fisher = Fisher(likelihood)
    fishers3.append(fisher(**params))

print(np.array([fisher.std(params='qiso') for fisher in fishers3]) / np.array([fisher.std(params='qiso') for fisher in fishers2]))

[1.02492723 1.02537954 1.02593978 1.02674348 1.03091388]


## BAO + full shape forecasts

In [6]:
cosmo = DESI()
fo = cosmo.get_fourier()

fishers_bao, fishers_bao_highk, fishers_fs_lowk, fishers_bao_fs = [], [], [], []
for zrange, nbar in list(zip(zranges, nbars)):
    z = np.mean(zrange)
    r = 0.5
    sigmaper = r * 12.4 * 0.758 * fo.sigma8_z(z, of='delta_cb') / 0.9
    f = fo.sigma8_z(z, of='theta_cb') / fo.sigma8_z(z, of='delta_cb')
    b1 = b0 * fo.sigma8_cb / fo.sigma8_z(z, of='delta_cb')
    params = {'b1': b1, 'sigmapar': (1. + f) * sigmaper, 'sigmaper': sigmaper}
    footprint = CutskyFootprint(area=14000., zrange=zrange, nbar=nbar * np.diff(zrange), cosmo=cosmo)
    template = BAOPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    #theory = DampedBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    theory = SimpleBAOWigglesTracerPowerSpectrumMultipoles(template=template)
    #for param in theory.params.select(basename='al*'): param.update(fixed=True)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.5))
    fisher = Fisher(likelihood)
    fisher_bao = fisher(**params)
    fishers_bao.append(fisher_bao)
    
    likelihood.init.update(klim=(0.1, 0.5))
    fisher = Fisher(likelihood)
    fishers_bao_highk.append(fisher(**params))
    
    #likelihood.init.update(klim=(0.01, 0.1))
    template = StandardPowerSpectrumTemplate(z=z, fiducial='DESI', apmode='qparqper')
    theory = SimpleTracerPowerSpectrumMultipoles(template=template)
    theory.params['sn0'].update(value=0., fixed=True)
    likelihood = SNWeightedPowerSpectrumLikelihood(theories=theory, data=params, footprints=footprint, klim=(0.01, 0.1))
    fisher = Fisher(likelihood)
    fisher_fs = fisher(**params)
    fishers_fs_lowk.append(fisher_fs)
    fishers_bao_fs.append(fisher_bao.view(params=['qpar', 'qper']) + fisher_fs)

In [7]:
# 20% - 30% improvement using the full shape < 0.1 h/Mpc
for fisher_bao, fisher_bao_fs in zip(fishers_bao, fishers_bao_fs):
    print(fisher_bao.std('qpar') / fisher_bao_fs.std('qpar'), fisher_bao.std('qper') / fisher_bao_fs.std('qper'))

1.2725654025771507 1.2324896132151342
1.272367128542189 1.2373296400618436
1.274587743678334 1.2441564082738958
1.2811731122821202 1.2550181393182207
1.334550532386808 1.3136720157505868
